In [2]:
import torch
import numpy as np

In [15]:
# Random seed to make results deterministic and reproducible
torch.manual_seed(0)

In [16]:
sentence = ("if you want to build a ship, don't drum up people together to"
            "collect wood and don;t assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [17]:
char_set = list(set(sentence))
char_dic = {c: i for i, c in enumerate(char_set)}

In [18]:
sequence_length = 10
hidden_size = len(char_dic)
dic_size = len(char_dic)
learning_rate = 0.1

In [19]:
x_data = []
y_data = []
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i+sequence_length]
    y_str = sentence[i+1:i+sequence_length+1]
    print(i, x_str, '->', y_str)
    x_data.append([char_dic[c] for c in x_str])
    y_data.append([char_dic[c] for c in y_str])

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [20]:
x_one_hot = [np.eye(dic_size)[x] for x in x_data]

In [21]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [22]:
# 모델 구성
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

model = Net(dic_size, hidden_size, 2)

In [23]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), learning_rate)

In [26]:
for epoch in range(100):
    outputs = model(X)
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0:
            predict_str += ''.join([char_set[t] for t in result])
        else:
            predict_str += char_set[result[-1]]

    print(predict_str)

dmmmsdmmmdmmmmmddddfmdddmdddmmmdmmmmdddmddmdmmdmmddmmmddmcmmmmmddmmmmsmsmmddmmmmdmmmddddmdmmmdddmddmmmddmmsmmmdmddmmdmdmdcmmmdmmddmdddmmmdmdmmdmmmmmddddmdmddmdmdmdddmddmdmmmdddmd
 wwko wkooawotwokmooawoaoommooa oowoaawotmaooowoowosoo;mowwo okooammtoomotoowaoo o;woowomotomowwotowmoooaaoowosoa mtoaoaawto womtowawto wokmtooowwooawoooaoowwoawaoaoo toooomowooo
e       e   e                           e               n  e     n   o        e                 e     e   e     e   e n o  e     e o  e   n       e              e     e   e      
ol,bol;ot,ath,iodpp;ea;ot,aiiaalaaie,gatpdeg;oiotie,iaapp;ao,inip;eadddsaip; opdlaaap;lappiaa;aap;ioiad;iaap,aao;dotaetiotao,;niip ;aaop;o;;aehd;aatpso;iaoiaepaippiipeo;dapinspo,
otatottot  tottto othtt tottho t  ttttottttth t tttt ttt tht  thttthttt h htttot  httttotttth thttttottnhthh  httnttttot  ttttthttththttttotttot thtott ttttttotto  thhto ttttto t
ot     t    t  o   o t      tn   tt    tn      to t     t  o        to   tt   t    tn      tn  to    tt  